# SQL Parser using Lark
### Lark
- Ref: [Lark Documentation](https://lark-parser.readthedocs.io/en/latest/)
  - Repo: [lark-parser](https://github.com/lark-parser/lark)

### Lark SQL Parser
- Ref: [sql_to_ibis](https://github.com/zbrookle/sql_to_ibis)
- Ref: [ibis](https://github.com/ibis-project/ibis)
  - Docs: https://ibis-project.org/docs/dev/

`sql_to_ibis` is a [Python](https://www.python.org/) package that translates SQL syntax into [ibis](https://github.com/ibis-project/ibis) expressions. This provides the capability of using only one SQL dialect to target many different backends.

- Ref: From `sql_to_ibis` project `sql_select_query.py`
- Ref: SQL grammar file <https://github.com/zbrookle/sql_to_ibis/tree/main/sql_to_ibis/grammar>

## Transform into Firebase Insert
### Transformers
* **Transformers** -  work bottom-up (or depth-first), starting with visiting the leaves and working their way up until ending at the root of the tree.
  * For each node visited, the transformer will call the appropriate method (callbacks), according to the node’s `data`, and use the returned value to replace the node, thereby creating a new tree structure.
  * Transformers can be used to implement map & reduce patterns. Because nodes are reduced from leaf to root, at any point the callbacks may assume the children have already been transformed

In [1]:
from firesql.firebase import FirebaseClient
from firesql.sql.sql_fire_client import FireSQLClient

firebaseClient = FirebaseClient()
firebaseClient.connect(credentials_json='../credentials/credentials.json')

# create FireSQLClient that is using the FireSQLAbstractClient interface.
client = FireSQLClient(firebaseClient)

In [2]:
import os
from lark import Lark, tree

_ROOT = "../firesql/sql"
GRAMMAR_PATH = os.path.join(_ROOT, "grammar", "firesql.lark")
with open(file=GRAMMAR_PATH) as sql_grammar_file:
    _GRAMMAR_TEXT = sql_grammar_file.read()
parser = Lark(_GRAMMAR_TEXT, parser="lalr")

In [3]:
from firesql.sql.sql_transformer import SelectTransformer

company = 'bennycorp'

sql1 = """
  INSERT INTO Companies/{}/Users (email, state)
    VALUES ('btscheung+test1@gmail.com', 'ACTIVE')
""".format(company)

ast = parser.parse(sql1)
statement = SelectTransformer().transform(ast)
statement

SQL_Insert(table=SQL_SelectFrom(part='Companies/bennycorp/Users', alias=None), columns=[SQL_ColumnRef(table=None, column='email', func=None), SQL_ColumnRef(table=None, column='state', func=None)], values=[SQL_ValueString(value='btscheung+test1@gmail.com'), SQL_ValueString(value='ACTIVE')])

In [4]:
from firesql.sql import SQLFireInsert

sqlCommand = SQLFireInsert()
sqlCommand.generate(statement, options={})

True

In [6]:
print(sqlCommand.part)
print(sqlCommand.columns)
print(sqlCommand.values)

Companies/bennycorp/Users
['email', 'state']
['btscheung+test1@gmail.com', 'ACTIVE']
